#web scraping

####Install required packages

In [1]:
!pip install firecrawl anthropic beautifulsoup4 python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 12.9 MB/s eta 0:00:00


###Import required libraries

In [3]:
import os
from bs4 import BeautifulSoup
from datetime import datetime
import re
from firecrawl import FirecrawlApp
import anthropic
from getpass import getpass

###API keys
#####- [Anthropic API](https://www.anthropic.com) (API key required)
#####- [Firecrawl API](https://www.firecrawl.com) (API key required)

In [4]:
os.environ["FIRECRAWL_API_KEY"] = getpass("Enter your FIRECRAWL API key: ")
os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your ANTHROPIC API key: ")

Enter your FIRECRAWL API key: ··········
Enter your ANTHROPIC API key: ··········


###Define the article URL to scrape

In [5]:
url = "https://www.aljazeera.net/sport/2025/4/18/barca-arsenal"

###Initialize Firecrawl and Anthropic clients using API keys

In [6]:
firecrawl_key = os.environ["FIRECRAWL_API_KEY"]
anthropic_key = os.environ["ANTHROPIC_API_KEY"]

client = anthropic.Client(api_key=anthropic_key)
app = FirecrawlApp(api_key=firecrawl_key)

###Scrape the URL content using Firecrawl

In [12]:
scrape_result = app.scrape_url(url, formats=['html'])

# Check if scraping succeeded
if not scrape_result:
    print("Failed to fetch data from the URL.")
else:
    print("Page content fetched successfully.")

Page content fetched successfully.


###Functions to extract title and main content from HTML

In [8]:
def extract_title_from_html(html: str) -> str:
    soup = BeautifulSoup(html, 'html.parser')
    title_tag = soup.find('title')
    if title_tag:
        return title_tag.get_text(strip=True)
    return "No Title Found"

def extract_main_content_from_html(html: str) -> str:
    soup = BeautifulSoup(html, 'html.parser')

    # Remove script and style elements
    for script in soup(["script", "style"]):
        script.decompose()

    # Try to locate the main content
    main_content = soup.select_one('main#main-content-area')

    if main_content:
        # Clean known unwanted blocks
        for unwanted in main_content.select('.article-info-block, .disclaimer-text, .article-author, .article-dates'):
            unwanted.decompose()

        # Extract paragraphs
        content = [p.get_text(strip=True) for p in main_content.find_all('p')]
        return '\n'.join(content)

    return "No Main Content Found"

###Extract title & content, display them, and save to a .txt file

In [11]:
if scrape_result:
    html = scrape_result.html

    title = extract_title_from_html(html)
    main_content = extract_main_content_from_html(html)

    # Show title and main content before saving
    print(f"Title:\n{title}\n")
    print(f"Main Content:\n{main_content}\n")

    # Save to a .txt file
    output_path = 'output.txt'
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(f"Title:\n{title}\n\n")
        f.write(f"Main Content:\n{main_content}\n")

    print(f"Title and main content saved to {output_path}")
else:
    print("No scrape result to process.")

Title:
إغلاق تلميح الأدوات

Main Content:
مع اكتمال عقد الفرق المتأهلة إلى نصف نهائي دوري أبطال أوروبا لكرة القدم، كشف حاسوب عملاق عن توقعاته لحظوظ الفرق الأربعة في التتويج باللقب.
وبلغت أندية كل من برشلونة وإنتر ميلان وأرسنال وباريس سان جيرمان الدور نصف النهائي من المسابقة الأوروبية العريقة بعد تجاوزها عقبة كل من بوروسيا دورتموند وبايرن ميونخ وريال مدريد وأستون فيلا على الترتيب.

وبحسب صحيفة "سبورت" الإسبانية فإن النسب المئوية التي حددها حاسوب "أوبتا" العملاق قد أثارت جدلا واسعا على مواقع التواصل الاجتماعي خاصة بعد أن وضع برشلونة في ذيل قائمة الفرق المرشحة للتتويج باللقب.
ويُعتبر أرسنال المرشح الأبرز لتحقيق اللقب بعد أن أقصى ريال مدريد حامل اللقب بنتيجة 5-1 في مجموع مباراتي الذهاب والإياب، بنسبة 28.7%.
اللافت أن هذه المرة الأولى التي يتصدر فيها أرسنال توقعات الذكاء الاصطناعي للفوز بالكأس "ذات الأذنين" هذا الموسم.

وحل إنتر ميلان ثانيا بعد بلوغه المربع الذهبي على حساب بايرن ميونخ بعد مباراتين قويتين مثيرتين، حيث بلغت نسبة تتويجه بدوري الأبطال وفق حاسوب "أوبتا" 25.5%.
ومنح الحاسوب العمل